In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

In [4]:
df = pd.read_csv('data_banknote_authentication.txt', index_col=[0]) 
# ou avec les séparateurs
# df= ruspini pd.read_csv('ruspini.csv', sep="," ,index_col=[0])

In [5]:
df
#df.iloc[[0]]

,8.6661,-2.8073,-0.44699,0
3.6216,,,,
4.54590,8.16740,-2.4586,-1.46210,0
3.86600,-2.63830,1.9242,0.10645,0
3.45660,9.52280,-4.0112,-3.59440,0
0.32924,-4.45520,4.5718,-0.98880,0
4.36840,9.67180,-3.9606,-3.16250,0
...,...,...,...,...
0.40614,1.34920,-1.4501,-0.55949,1
-1.38870,-4.87730,6.4774,0.34179,1
-3.75030,-13.45860,17.5932,-2.77710,1


In [6]:
df.fillna(df.median())

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,8.6661,-2.8073,-0.44699,0
3.6216,,,,
4.54590,8.16740,-2.4586,-1.46210,0
3.86600,-2.63830,1.9242,0.10645,0
3.45660,9.52280,-4.0112,-3.59440,0
0.32924,-4.45520,4.5718,-0.98880,0
4.36840,9.67180,-3.9606,-3.16250,0
...,...,...,...,...
0.40614,1.34920,-1.4501,-0.55949,1
-1.38870,-4.87730,6.4774,0.34179,1
-3.75030,-13.45860,17.5932,-2.77710,1


In [7]:
df.fillna(df.mean())

,8.6661,-2.8073,-0.44699,0
3.6216,,,,
4.54590,8.16740,-2.4586,-1.46210,0
3.86600,-2.63830,1.9242,0.10645,0
3.45660,9.52280,-4.0112,-3.59440,0
0.32924,-4.45520,4.5718,-0.98880,0
4.36840,9.67180,-3.9606,-3.16250,0
...,...,...,...,...
0.40614,1.34920,-1.4501,-0.55949,1
-1.38870,-4.87730,6.4774,0.34179,1
-3.75030,-13.45860,17.5932,-2.77710,1


In [ ]:
import prince
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

def cleaning(df):
    #df is a csv file
#Cleaning kidney 

    #Replace missing number data by 0
    #Replace the other type of value by "Unk"
    for col in list(df.columns):
        if df[col].dtype==int or df[col].dtype==float:
            df[col]=df[col].fillna(0)
        else : 
            df[col]=df[col].fillna('Unk')

def class_column(df):
    #df is a csv file 
#Extract the class column of a dataframe 

    return df.iloc[:,-1:]

def type_data(df):
    #Return the information of the type of data in the DataFrame in order to now which method to use :
        #If only numbers : PCA (2)
        #If only categorial data : MCA (1)
        #Mixed data : FAMD (3)
        
    if (df.select_dtypes(include = 'float64')).size==0 or (df.select_dtypes(include = 'int64')).size==0 :
        #Case where there is only categorical data
        return 1
    if (df.select_dtypes(include = 'object')).size==0:
        #Case where ther is only quantitative data
        return 2
    else :
        #Case of a mixed type of data
        return 3
        

def famd(df):
    #df is a csv file which has already been cleaned : there is no Nan value
    
    ####TO CHANGE###
    cleaning(df)
    ####
    
    famd=prince.FAMD(n_components=2, #We chose to have a 2D representation
                 n_iter=10,
                 copy=True,
                 check_input=True,
                 engine='auto',
                 random_state=42
                )
    
    df_feat=df.drop(df.columns[-1], axis='columns')
    famd=famd.fit(df_feat)
    print(famd.row_coordinates(df_feat))
    ax = famd.plot_row_coordinates(
     df_feat,
     ax=None,
     figsize=(6, 6),
     x_component=0,
     y_component=1,
     color_labels=list(encoded_class(df)),
     ellipse_outline=False,
     ellipse_fill=True,
     show_points=True)
    return ax.get_figure()

def pca(df):
    #df is a DataFrame which have already been cleaned
    
    ####TO CHANGE###
    cleaning(df)
    ####
    
    #Definition of the model 
    pca = prince.PCA(
     n_components=2,
     n_iter=10,
     rescale_with_mean=True,
     rescale_with_std=True,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42) 
    
    #We drop the class column from the DataFrame
    
    df_feat=df.drop(df.columns[-1], axis='columns')
    
    #We train the model according to our dataframe 
    pca=pca.fit(df_feat)
    
    #Apply the model to the DataFrame
    pca.transform(df_feat).head()
    
    #Plot the result
    ax = pca.plot_row_coordinates(
     df_feat,
     ax=None,
     figsize=(6, 6),
     x_component=0,
     y_component=1,
     labels=None,
     color_labels=y,
     ellipse_outline=False,
     ellipse_fill=True,
     show_points=True)


def encoded_class(df):
    #Take the class column of the dataframe and return a culumn of the same size,
    #ordered in the same way but where categories are not strings but integers
    c=class_column(df)
    print(type(c))
    labelencoder = LabelEncoder()# Assigning numerical values and storing in another column
    c.iloc[:,0]=labelencoder.fit_transform(c.iloc[:,-1])
    return c.iloc[:,0]